<a href="https://colab.research.google.com/github/Mohammadhsiavash/Sentiment-analysis-with-NLP/blob/main/inferentia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install bertviz
!pip install datasets
!pip install transformers[torch]
!pip install accelerate
!pip install huggingface_hub
!pip install sentencepiece
!pip install shap
!pip install lime

In [ ]:
from huggingface_hub  import login
login()

In [ ]:
from matplotlib import pyplot as plt
import spacy
from wordcloud import WordCloud

from transformers import AlbertForSequenceClassification, AlbertTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
import numpy as np
import torch


def create_word_cloud(series):
    # Combine all text from the series into a single string
    text = ' '.join(series.tolist())

    # Load spaCy's English language model
    nlp = spacy.load('en_core_web_sm')

    # Create a set of spaCy's English stop words
    stopwords = set(nlp.Defaults.stop_words)

    # Step 4: Create an instance of WordCloud and generate the word cloud
    wordcloud = WordCloud(stopwords=stopwords).generate(text)

    # Step 6: Display the word cloud using matplotlib
    plt.figure(figsize=(8, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

import re

def remove_mental_health_references(df):
  unique_labels = list(df.subreddit.unique())
  unique_labels.extend(list(map(lambda x: x.lower(), unique_labels)))
  unique_labels.extend(list(map(lambda x: x.upper(), unique_labels)))
  unique_labels = set(unique_labels)
  pattern = '|'.join(unique_labels)
  df.body = df.body.str.replace(pattern, '', regex=True)
  return df


def not_none(example):
    return example['body'] is not None

def prepare_dataframe(df):
  # Concatenate title and body
  df['body'] = df.body.fillna('')
  df['body'] = df.body.str.cat(df.title, sep=' ')

  # Removed deleted posts
  df = df[~df.author.str.contains('\[deleted\]')]
  df = df[~df.body.str.contains('\[removed\]')]
  df = df[~df.body.str.contains('\[deleted\]')]
  df = df[~df.body.str.contains('\[deleted by user\]')]

  # Removed moderador posts
  df = df[df.author!='AutoModerator']

  return df[['body', 'subreddit']]


In [ ]:
dataset = load_dataset("solomonk/reddit_mental_health_posts")
df = dataset["train"].to_pandas()
df = prepare_dataframe(df).reset_index(drop=True)
df = df.pipe(remove_mental_health_references)
dataset = Dataset.from_pandas(df)
dataset_sampled = dataset.train_test_split(test_size=0.7, seed=42)['train']

train_val_test = dataset_sampled.train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_test['train']
test_val_dataset = train_val_test['test']

test_val_split = test_val_dataset.train_test_split(test_size=0.5, seed=42)
validation_dataset = test_val_split['train']
test_dataset = test_val_split['test']

In [ ]:
le = LabelEncoder()
le.fit(dataset_sampled['subreddit'])

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

# This is the inferential of the model

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = AlbertForSequenceClassification.from_pretrained("Frorozcol/albert_classification_reddit",  output_attentions=True).cuda()

In [ ]:
!pip install transformers-interpret

In [ ]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer,
    custom_labels =  le.classes_)
word_attributions = cls_explainer(test_dataset["body"][1519], class_name="OCD")

In [ ]:
cls_explainer.visualize("distilbert_viz.html")

In [ ]:
word_attributions = cls_explainer(test_dataset["body"][2034], class_name="ADHD")
cls_explainer.visualize("distilbert_viz.html")

In [ ]:
cls_explainer